In [ ]:
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
import copy
import pickle


In [ ]:
# Config variables
clean_data_path = os.path.join("..", "clean_data") 

In [ ]:
df_player = pd.read_csv(os.path.join(clean_data_path, "player.csv"))
df_player = df_player.loc[:, ~df_player.columns.str.contains('^Unnamed')]

In [ ]:
df_ball = pd.read_csv(os.path.join(clean_data_path, "ball.csv"))
df_ball = df_ball.loc[:, ~df_ball.columns.str.contains('^Unnamed')]

In [ ]:
empty_match_obj = {
    "runs_scored" : {},
    "balls_faced" : {},
    "dismissed" : False,
    "dismissed_by_type" : "",
    "num_4s" : 0,
    "num_6s" : 0,
    "batting_innings" : 0,
    "wickets_taken" : {
        "Left-hand bat" : 0,
        "Right-hand bat" : 0 
    },
    "wickets_with_bonus" : 0,
    "balls_bowled" : {
        "Left-hand bat" : 0,
        "Right-hand bat" : 0 
    },
    "runs_given" : 0,
    "maidens_bowled" : 0,
    "catches" : 0,
    "direct_runouts" : 0,
    "indirect_runouts" : 0,
    "stumping" : 0,
    "fantasy_points" : 0
}

In [ ]:
bowler_dismissal_type = ['bowled',
 'caught',
 'caught and bowled',
 'hit wicket',
 'lbw',
 'stumped']

In [ ]:
bonus_bowler_dismissal_type = ['bowled', 'lbw']

In [ ]:
bowling_styles = set(df_player["bowling_style"])
empty_runs_scored_obj= {}
for bowling_style in bowling_styles:
    if type(bowling_style) != str:
        continue
    empty_runs_scored_obj[bowling_style] = 0
empty_match_obj["runs_scored"] = copy.deepcopy(empty_runs_scored_obj)
empty_match_obj["balls_faced"] = copy.deepcopy(empty_runs_scored_obj)

In [ ]:
fantasy_obj = {}

In [ ]:
over_number = 0
runs_given_in_over = 0

for i,row in tqdm(df_ball.iterrows()):
    match_id = int(row["match_id"])
    try:
        fantasy_obj[row["batsman"]]
    except Exception as e:
        fantasy_obj[row["batsman"]] = {}
    
    try: 
        fantasy_obj[row["batsman"]][match_id]
    except Exception as e:
        fantasy_obj[row["batsman"]][match_id] = copy.deepcopy(empty_match_obj)
        
    try:
        fantasy_obj[row["bowler"]]
    except Exception as e:
        fantasy_obj[row["bowler"]] = {}
    
    try: 
        fantasy_obj[row["bowler"]][match_id]
    except Exception as e:
        fantasy_obj[row["bowler"]][match_id] = copy.deepcopy(empty_match_obj)
        
    try:
        if row["player_dismissed"] >= 0:
            fantasy_obj[int(row["player_dismissed"])]
    except Exception as e:
        fantasy_obj[int(row["player_dismissed"])] = {}
    try:
        if row["player_dismissed"] >= 0:
            fantasy_obj[int(row["player_dismissed"])][match_id]
    except Exception as e:
        fantasy_obj[int(row["player_dismissed"])][match_id] = copy.deepcopy(empty_match_obj)
    if row["player_dismissed"] >= 0:
        fantasy_obj[int(row["player_dismissed"])][match_id]["dismissed"] = True

    batsman = df_player.loc[df_player['player_id'] == row["batsman"]]
    bowler = df_player.loc[df_player['player_id'] == row["bowler"]]
    fielders = []
    if type(row["fielders"]) == str:
        fielders = row["fielders"].split(",")
    
    #batsman
    batsman_runs = row["batsman_runs"]
    if row["wide_runs"] == 0:
        fantasy_obj[row["batsman"]][match_id]["balls_faced"][bowler["bowling_style"].values[0]] += 1
    
    fantasy_obj[row["batsman"]][match_id]["runs_scored"][bowler["bowling_style"].values[0]] += batsman_runs
    if batsman_runs == 6:
        fantasy_obj[row["batsman"]][match_id]["num_6s"] += 1
    elif batsman_runs == 4:
        fantasy_obj[row["batsman"]][match_id]["num_4s"] += 1
    if fantasy_obj[row["batsman"]][match_id]["batting_innings"] == 0:
        fantasy_obj[row["batsman"]][match_id]["batting_innings"] = row["innings_number"]

    #bowler
    if int(row["ball_number"]) != over_number:
        over_number = int(row["ball_number"])
        runs_given_in_over = 0
    if row["bye_runs"] == 0 and row["legbye_runs"] == 0:
        runs_given_in_over += row["total_runs"]
    ball_number = (row["ball_number"] % 1) * 10
    if ball_number == 6 and runs_given_in_over == 0:
        fantasy_obj[row["bowler"]][match_id]["maidens_bowled"] += 1

    if row["dismissal_type"] in bowler_dismissal_type:
        fantasy_obj[row["bowler"]][match_id]["wickets_taken"][batsman["batting_style"].values[0]] += 1
        fantasy_obj[row["batsman"]][match_id]["dismissed_by_type"] = bowler["bowling_style"].values[0]
    if row["dismissal_type"] in bonus_bowler_dismissal_type:
        fantasy_obj[row["bowler"]][match_id]["wickets_with_bonus"] += 1
    if row["wide_runs"] == 0 and row["noball_runs"] == 0:
        fantasy_obj[row["bowler"]][match_id]["balls_bowled"][batsman["batting_style"].values[0]] += 1
    if row["bye_runs"] == 0 and row["legbye_runs"] == 0:
        fantasy_obj[row["bowler"]][match_id]["runs_given"] += row["total_runs"]
    if row["dismissal_type"] == "caught and bowled":
        fantasy_obj[row["bowler"]][match_id]["catches"] += 1
        
    if fantasy_obj[row["bowler"]][match_id]["batting_innings"] == 0:
        fantasy_obj[row["bowler"]][match_id]["batting_innings"] = (row["innings_number"] + 1) % 2
        
    #fielder
    for fielder in fielders:
        fielder = int(fielder)
        try:
            fantasy_obj[fielder]
        except Exception as e:
            fantasy_obj[fielder] = {}
        try: 
            fantasy_obj[fielder][match_id]
        except Exception as e:
            fantasy_obj[fielder][match_id] = copy.deepcopy(empty_match_obj)
        
        if fantasy_obj[fielder][match_id]["batting_innings"] == 0:
            fantasy_obj[fielder][match_id]["batting_innings"] = (row["innings_number"] + 1) % 2
        
        if row["dismissal_type"] == "run out":
            if len(fielders) == 1:
                fantasy_obj[fielder][match_id]["direct_runouts"] += 1
            else:
                fantasy_obj[fielder][match_id]["indirect_runouts"] += 1
        elif row["dismissal_type"] == "stumped":
            fantasy_obj[fielder][match_id]["stumping"] += 1
        else:
            fantasy_obj[fielder][match_id]["catches"] += 1

In [ ]:
for player in fantasy_obj:
    for match in fantasy_obj[player]:
        fantasy_points = 0
        #batting points
        runs_scored = 0
        for bowling_style in fantasy_obj[player][match]["runs_scored"]:
            runs_scored += fantasy_obj[player][match]["runs_scored"][bowling_style]
        fantasy_points += runs_scored
        if runs_scored >= 100:
            fantasy_points += 16
        elif runs_scored >= 50:
            fantasy_points += 8
        elif runs_scored >= 30:
            fantasy_points += 4
        elif runs_scored == 0 and fantasy_obj[player][match]["dismissed"] == True:
            fantasy_points -= 2
        fantasy_points += (fantasy_obj[player][match]["num_6s"] * 2)
        fantasy_points += fantasy_obj[player][match]["num_4s"]
        balls_faced = sum(fantasy_obj[player][match]["balls_faced"].values())
        if balls_faced >= 10:
            strike_rate = (runs_scored * 100)/balls_faced
            if strike_rate > 170:
                fantasy_points += 6
            elif strike_rate > 150:
                fantasy_points += 4
            elif strike_rate >= 130:
                fantasy_points += 2
            elif strike_rate < 50:
                fantasy_points -= 6
            elif strike_rate < 60:
                fantasy_points -= 4
            elif strike_rate <= 70:
                fantasy_points -= 2

        #bowling points
        wickets_taken = 0
        for batting_style in fantasy_obj[player][match]["wickets_taken"]:
            wickets_taken += fantasy_obj[player][match]["wickets_taken"][batting_style]
        fantasy_points += (wickets_taken * 25)
        fantasy_points += (fantasy_obj[player][match]["wickets_with_bonus"] * 8)
        fantasy_points += (fantasy_obj[player][match]["maidens_bowled"] * 12)
        if wickets_taken == 3:
            fantasy_points += 4
        elif wickets_taken == 4:
            fantasy_points += 8
        elif wickets_taken >= 5:
            fantasy_points += 16
        balls_bowled = sum(fantasy_obj[player][match]["balls_bowled"].values())
        if balls_bowled >= 12:
            economy = (fantasy_obj[player][match]["runs_given"] * 6)/balls_bowled
            if economy < 5:
                fantasy_points += 6
            elif economy < 6:
                fantasy_points += 4
            elif economy < 7:
                fantasy_points += 2
            elif economy > 12:
                fantasy_points -= 6
            elif economy > 11:
                fantasy_points -= 4
            elif economy >= 10:
                fantasy_points -= 2
        
        #fielding points
        fantasy_points += (fantasy_obj[player][match]["catches"] * 8)
        if fantasy_obj[player][match]["catches"] >= 3:
            fantasy_points += 4
        fantasy_points += (fantasy_obj[player][match]["stumping"] * 12)
        fantasy_points += (fantasy_obj[player][match]["direct_runouts"] * 12)
        fantasy_points += (fantasy_obj[player][match]["indirect_runouts"] * 6)
        
        fantasy_obj[player][match]["fantasy_points"] = fantasy_points

In [ ]:
with open(os.path.join(clean_data_path, 'fantasy.pkl'), 'wb') as fantasy_pi:
    pickle.dump(fantasy_obj, fantasy_pi, pickle.HIGHEST_PROTOCOL)